<a href="https://colab.research.google.com/github/Ramykaz/Solar-Panel-Anomaly-Detection/blob/main/RGB_TRAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== INSTALL DEPENDENCIES =====
!pip install roboflow ultralytics --quiet

# ===== VERIFY GPU =====
import torch
print(f"GPU available: {torch.cuda.is_available()}")
print(f"GPU device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# ===== DOWNLOAD DATASET =====
from roboflow import Roboflow
rf = Roboflow(api_key="MKleN50baOrRxLo0qCI3")
project = rf.workspace("mine-rok5l").project("deneme-2-ipque-3txzv")
dataset = project.version(1).download("yolov8")

# ===== VALID YOLOv8 AUGMENTATIONS CONFIG =====
augmentation_config = {
    # Color & Contrast (VALID parameters)
    "hsv_h": 0.03,        # Hue variation
    "hsv_s": 0.8,         # Saturation
    "hsv_v": 0.5,         # Value (brightness)

    # Geometric Transformations (VALID parameters)
    "degrees": 15.0,      # Rotation (+/- degrees)
    "translate": 0.2,     # Translation (± fraction)
    "scale": 0.3,         # Scale augmentation
    "shear": 8.0,         # Shear transformation
    "perspective": 0.001, # Perspective transformation
    "flipud": 0.5,        # Vertical flip probability
    "fliplr": 0.5,        # Horizontal flip probability

    # Advanced Augmentations (VALID parameters)
    "mosaic": 0.8,        # Mosaic augmentation probability
    "mixup": 0.2,         # Mixup augmentation probability
    "copy_paste": 0.2,    # Copy-paste augmentation
}

# ===== PREPROCESSING CONFIG =====
preprocessing_config = {
    "imgsz": 640,                 # Input image size
    "rect": False,                # Rectangular training
    "cache": "ram",               # Cache dataset in RAM
    "single_cls": False,          # Multi-class training
    "mask_ratio": 4,              # Mask downsampling ratio
    "overlap_mask": True,         # Allow masks to overlap
}

# ===== TRAINING CONFIG =====
training_config = {
    "epochs": 50,                # More epochs for better convergence
    "batch": 16,                  # Batch size
    "workers": 8,                 # Data loading workers
    "device": "0",                # Use GPU
    "optimizer": "AdamW",         # Better optimizer
    "lr0": 0.001,                 # Initial learning rate
    "lrf": 0.01,                  # Final learning rate
    "weight_decay": 0.0005,       # Regularization
    "momentum": 0.9,              # SGD momentum
    "warmup_epochs": 3,           # Learning rate warmup
    "warmup_momentum": 0.8,       # Warmup momentum
    "warmup_bias_lr": 0.1,        # Warmup bias learning rate
    "box": 7.5,                   # Box loss gain
    "cls": 0.5,                   # Class loss gain
    "dfl": 1.5,                   # Distribution Focal Loss gain
    "pose": 12.0,                 # Pose loss gain (for segmentation)
    "kobj": 1.0,                  # Keypoint object loss gain
    "nbs": 64,                    # Nominal batch size
}

# ===== MODEL SETUP & TRAINING =====
from ultralytics import YOLO
import os

# Load segmentation model
model = YOLO("yolov8m-seg.pt")  # Medium model for good balance

# Combine all configurations (only valid parameters)
train_args = {
    "data": f"{dataset.location}/data.yaml",
    "name": "solar_defects_v2_5class",
    "project": "/content/runs",
    "exist_ok": True,
    **augmentation_config,
    **preprocessing_config,
    **training_config
}

# Start training
print("🚀 Starting training with 5 classes...")
results = model.train(**train_args)

# ===== VALIDATION =====
print("📊 Validating model...")
metrics = model.val()
print(f"✅ Validation mAP50-95: {metrics.box.map:.3f}")
print(f"✅ Mask mAP50-95: {metrics.seg.map:.3f}")

# ===== EXPORT =====
print("💾 Exporting model...")
model.export(format="onnx", dynamic=True, simplify=True)
print("✅ Export complete!")

# ===== TEST PREDICTION =====
print("🧪 Testing prediction...")
import cv2
import numpy as np

# Test on a validation image
test_image_path = f"{dataset.location}/valid/images/"
test_files = [f for f in os.listdir(test_image_path) if f.endswith(('.jpg', '.png'))]
if test_files:
    test_img = f"{test_image_path}/{test_files[0]}"
    results = model.predict(test_img, conf=0.4, save=True)
    print(f"✅ Test prediction saved: {results[0].save_dir}")

# ===== SAVE RESULTS =====
from google.colab import files
import zipfile

# Zip and download results
print("📦 Preparing results for download...")
results_dir = "/content/runs/segment/solar_defects_v2_5class"
zip_path = "/content/training_results.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(results_dir):
        for file in files:
            zipf.write(os.path.join(root, file),
                      os.path.relpath(os.path.join(root, file), results_dir))

files.download(zip_path)
print("🎉 Training complete! Results downloaded.")

GPU available: True
GPU device: Tesla T4
loading Roboflow workspace...
loading Roboflow project...
🚀 Starting training with 5 classes...
Ultralytics 8.3.182 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Deneme-2-1/data.yaml, degrees=15.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.03, hsv_s=0.8, hsv_v=0.5, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m-seg.pt, momentum=0.9, mosaic=0.8, multi_scale=False, name=solar_defects_v2_

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

 18                  -1  2   1846272  ultralytics.nn.modules.block.C2f             [576, 384, 2]                 
 19                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  2   4207104  ultralytics.nn.modules.block.C2f             [960, 576, 2]                 
 22        [15, 18, 21]  1   5161919  ultralytics.nn.modules.head.Segment          [5, 32, 192, [192, 384, 576]] 
YOLOv8m-seg summary: 191 layers, 27,242,543 parameters, 27,242,527 gradients, 110.4 GFLOPs

Transferred 531/537 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2608.1±726.4 MB/s, size: 2114.2 KB)


train: Scanning /content/Deneme-2-1/train/labels... 1675 images, 242 backgrounds, 0 corrupt: 100%|██████████| 1675/1675 [00:00<00:00, 1832.23it/s]

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     86/150      7.69G      1.229      2.153      2.292      1.025        203        640: 100%|██████████| 105/105 [01:19<00:00,  1.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  1.76it/s]

                   all        201       1532      0.562      0.342      0.391      0.264      0.512      0.335      0.367      0.199



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


     87/150      7.71G      1.205      2.245      2.251      1.037        169        640:  15%|█▌        | 16/105 [00:11<01:07,  1.31it/s]